# 트리의 앙상블
## 정형 데이터와 비정형 데이터

정형 데이터: 구조화된 데이터. csv나 데이터베이스에 저장하기 쉬움   
비정형 데이터: 텍스트 데이터, 사진, 음악 등   

비구조화된 데이터를 저장하는 데이터베이스 : NoSQL이 대표적   

정형 데이터를 다루는 데 가장 뛰어난 성과를 보이는 알고리즘: 앙상블 학습(ensemble learning)   
비정형 데이터 -> 7장

## 랜덤포레스트(Random Forest)

앙상블 학습의 대표주자   
부트스트랩 샘플 : 중복을 허용하는 샘플 뽑기 방법

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [3]:
# 하이퍼파라미터 튜닝은 실시하지 않음(예제가 간단하고 특성이 많지 않아 앙상블 학습에 집중)
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier   # 결정 트리 대신 랜덤 포레스트 사용 - 결정 트리의 앙상블이기 때문에 결정 트리의 매개변수를 모두 제공

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [4]:
# 랜덤 포레스트의 장점: 특성 하나에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다.
# 과대 적합을 줄이고 일반화 성능을 높이는 데 도움이 된다.
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [6]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)
# 부트스트랩 샘플에 포함되지 않고 남은 샘플 = OOB(Out of bag)
# 이를 검증 세트의 역할로 사용할 수 있다.
# obb_score = True로 지정하여 결과값 프린트
# 교차 검증을 대신할 수 있어서, 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있다. obb_score_ 확인

0.8934000384837406


## 엑스트라트리

부트스트랩 샘플을 사용하지 않고, 전체 훈련 세트를 사용한다.    
노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할함.
하나의 결정 트리에서 특성을 무작위로 분할하면 성능이 낮아지지만,
많은 트리를 앙상블하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있다.

In [8]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [9]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [12]:
# 학습률을 증가시키고 트리의 개수를 늘리면 성능이 향상
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대 적합을 잘 억제함 # learning rate

0.9464595437171814 0.8780082549788999


In [13]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
# 특성 중요도 확인

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 그래디언트 부스팅

정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘    
먼저 입력 특성을 256개의 구간으로 나눔    
일반적으로 기본 매개변수에서 안정적인 성능을 얻을 수 있다.   
트리의 개수를 지정하는 데 n_estimators 대신 부스팅 반복 횟수를 지정하는 max_iter를 사용

In [14]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대적합을 잘 억제하면서 그레디언트 부스팅보다 조금 더 높은 성능

0.9321723946453317 0.8801241948619236


In [15]:
# 특성 중요도를 계산하기 위한 permutation_importance  함수 사용 : 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지 관찰,  어떤 특성이 중요한지 계산
# n_repeats 매개변수 = 랜덤하게 섞을 횟수 (기본값 5)
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [16]:
# 테스트 세트의 결과 확인
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [17]:
# 최종 성능
hgb.score(test_input, test_target)

0.8723076923076923

단일 결정 트리보다 좋은 결과를 보여줌   
참고: 히스토그램 기반 그레디언트 부스팅의 회귀 버전 = HistGradientBoostingRegressor

#### XGBoost


※ 사이킷런 외에 그레디언트 부스팅 알고리즘 구현한 라이브러리 = XGBoost
tree_method를 'hist'로 지정하여 사용

In [18]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


#### LightGBM

※ 또다른 라이브러리 = LightGBM

In [19]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
